In [1]:
from utils import measure_accuracy, read_jsonl, LABEL_NAMES_MAP

inference_val_file = '/Users/ryanarman/code/lab/banking77/notebooks/data/system_prompt_v2_sft_results_3072.jsonl'
inference_val_data = read_jsonl(inference_val_file)

accuracy_val, correct_val, total_val, errors_val, incorrect_list_val = measure_accuracy(inference_val_data)


inference_test_file = '/Users/ryanarman/code/lab/banking77/notebooks/data/full_fft_results_3066.jsonl'
inference_test_data = read_jsonl(inference_test_file)

accuracy_test, correct_test, total_test, errors_test, incorrect_list_test = measure_accuracy(inference_test_data)

inference_file_val_after_correction = '/Users/ryanarman/code/lab/banking77/notebooks/data/improved_train_results_v2_3112.jsonl'
inference_val_after_correction_data = read_jsonl(inference_file_val_after_correction)
accuracy_val_corrected, correct_val_corrected, total_val_corrected, errors_val_corrected, incorrect_list_val_corrected = measure_accuracy(inference_val_after_correction_data)

inference_file_test_after_correction = '/Users/ryanarman/code/lab/banking77/notebooks/data/improved_train_results_v2_3113.jsonl'
inference_test_after_correction_data = read_jsonl(inference_file_test_after_correction)
accuracy_test_corrected, correct_test_corrected, total_test_corrected, errors_test_corrected, incorrect_list_test_corrected = measure_accuracy(inference_test_after_correction_data)

Accuracy: 91.60% (1833/2001)
Accuracy: 93.51% (2880/3080)
Accuracy: 92.30% (1847/2001)
Accuracy: 93.44% (2878/3080)


In [2]:
print(f"val: {len(incorrect_list_val)},  val_corrected: {len(incorrect_list_val_corrected)}")
print(f"test: {len(incorrect_list_test)},  test_corrected: {len(incorrect_list_test_corrected)}")

val: 168,  val_corrected: 154
test: 200,  test_corrected: 202


In [3]:
from collections import Counter, defaultdict

def get_most_common_pairs(incorrect_list):
    wihout_index = [sorted([item[1], item[2]]) for item in incorrect_list]
    wihout_index_str = [f'{item[0]}-{item[1]}' for item in wihout_index]
    cnt = Counter(wihout_index_str)
    most_common_items = cnt.most_common(15)
    most_common_pairs = [item[0] for item in most_common_items]
    return cnt, most_common_pairs, most_common_items

def get_map_incorrect_pairs_to_idx(incorrect_list):
    d = defaultdict(list)

    for item in incorrect_list:
        idx = item[0]
        inference_class_label = item[1]
        gt_label = item[2]
        pair = sorted([inference_class_label, gt_label])
        pair_str = f'{pair[0]}-{pair[1]}'
        d[pair_str].append(idx)
    return d

# Validation 

In [7]:
pairs_count, most_common_pairs, most_common_items = get_most_common_pairs(incorrect_list_val)
pairs_count_corrected, most_common_pairs_corrected, most_common_items_corrected = get_most_common_pairs(incorrect_list_val_corrected)

print("class_name_1 <> class_name_2: count -> count_after_correction")
for  item in most_common_items:
    item_corrected = pairs_count_corrected[item[0]]
    class_label_1, class_label_2 = item[0].split('-')
    class_name_1 = LABEL_NAMES_MAP[int(class_label_1)]
    class_name_2 = LABEL_NAMES_MAP[int(class_label_2)]
    class_pair = f"{class_name_1} <> {class_name_2}"
    print(f"{class_pair:^{70}} : {item[1]} -> {item_corrected}")

class_name_1 <> class_name_2: count -> count_after_correction
                   pending_top_up <> top_up_failed                     : 8 -> 5
      balance_not_updated_after_bank_transfer <> transfer_timing       : 7 -> 5
              beneficiary_not_allowed <> failed_transfer               : 6 -> 6
                 pending_top_up <> topping_up_by_card                  : 5 -> 2
        pending_transfer <> transfer_not_received_by_recipient         : 5 -> 4
           card_payment_not_recognised <> compromised_card             : 4 -> 4
              lost_or_stolen_phone <> passcode_forgotten               : 4 -> 0
           card_payment_fee_charged <> transfer_fee_charged            : 4 -> 2
  card_payment_not_recognised <> direct_debit_payment_not_recognised   : 4 -> 6
              declined_card_payment <> declined_transfer               : 3 -> 4
       fiat_currency_support <> supported_cards_and_currencies         : 3 -> 3
       pending_cash_withdrawal <> wrong_amount_of_cash_rec

In [5]:
def print_incorrect_class_queires(incorrect_list, inference_data, most_common_items_dict):
    d = get_map_incorrect_pairs_to_idx(incorrect_list)
    for item in most_common_items_dict:
        class_label_1, class_label_2 = item[0].split('-')
        class_name_1 = LABEL_NAMES_MAP[int(class_label_1)]
        class_name_2 = LABEL_NAMES_MAP[int(class_label_2)]
        class_pair = f"{class_name_1} <> {class_name_2}"
        idx_list = d[item[0]]
        # print(class_label_1, class_label_2, idx_list)
        print(class_pair)
        for i, idx in enumerate(idx_list):
            query = inference_data[idx]['messages'][1]['content']
            pred_label = inference_data[idx]['messages'][2]['content']
            gt_label = inference_data[idx]['metadata']['label_name']
            print(f"query {i+1}: {query}, pred_label: {LABEL_NAMES_MAP[int(pred_label)]}, gt_label: {gt_label}")
        print('-'*60)

print_incorrect_class_queires(incorrect_list_val, inference_val_data, most_common_items)


pending_top_up <> top_up_failed
query 1: I added money but my top-up wasn't processed by the app., pred_label: pending_top_up, gt_label: top_up_failed
query 2: My top-up hasn't gone through, pred_label: top_up_failed, gt_label: pending_top_up
query 3: Why hasn't my top-up gone through?, pred_label: top_up_failed, gt_label: pending_top_up
query 4: The top-up is broken., pred_label: top_up_failed, gt_label: pending_top_up
query 5: It appears my top-up has not gone through., pred_label: top_up_failed, gt_label: pending_top_up
query 6: I added money to the card but the transaction didn't go through, pred_label: top_up_failed, gt_label: pending_top_up
query 7: I requested for a top-up but it has not reflected, pred_label: pending_top_up, gt_label: top_up_failed
query 8: Is there a problem with the top up system? My transaction hasn't gone through properly, pred_label: top_up_failed, gt_label: pending_top_up
------------------------------------------------------------
balance_not_updated_aft

In [6]:
print_incorrect_class_queires(incorrect_list_val_corrected, inference_val_after_correction_data, most_common_items_corrected)


beneficiary_not_allowed <> failed_transfer
query 1: I have tried to submit a transfer a couple of times now, and for some reason, it continues to be unsuccessful. Do you know why this may be?, pred_label: failed_transfer, gt_label: beneficiary_not_allowed
query 2: Why am I unable to transfer money when I was able to before?, pred_label: failed_transfer, gt_label: beneficiary_not_allowed
query 3: My attempted transfer keeps kicking back with an error message.  Could you please help me to resolve this?, pred_label: failed_transfer, gt_label: beneficiary_not_allowed
query 4: I have made several unsuccessful attempts to make a transfer.  I'm getting frustrated at getting error messages and it not going through., pred_label: failed_transfer, gt_label: beneficiary_not_allowed
query 5: Why is my transfer not working? I tried multiple times to transfer money, and all that happens is an error message pops up., pred_label: failed_transfer, gt_label: beneficiary_not_allowed
query 6: why didnt a t

# Test

In [25]:
pairs_count, most_common_pairs, most_common_items = get_most_common_pairs(incorrect_list_test)
pairs_count_corrected, most_common_pairs_corrected, most_common_items_corrected = get_most_common_pairs(incorrect_list_test_corrected)

print("class_name_1 <> class_name_2: count -> count_after_correction")
for  item in most_common_items:
    item_corrected = pairs_count_corrected[item[0]]
    class_label_1, class_label_2 = item[0].split('-')
    class_name_1 = LABEL_NAMES_MAP[int(class_label_1)]
    class_name_2 = LABEL_NAMES_MAP[int(class_label_2)]
    class_pair = f"{class_name_1} <> {class_name_2}"
    print(f"{class_pair:^{70}} : {item[1]} -> {item_corrected}")

class_name_1 <> class_name_2: count -> count_after_correction
     balance_not_updated_after_bank_transfer <> pending_transfer       : 8 -> 6
                card_arrival <> card_delivery_estimate                 : 7 -> 8
  card_payment_not_recognised <> direct_debit_payment_not_recognised   : 6 -> 6
              declined_card_payment <> declined_transfer               : 6 -> 6
      balance_not_updated_after_bank_transfer <> transfer_timing       : 6 -> 5
              exchange_via_app <> fiat_currency_support                : 4 -> 6
                 pending_top_up <> topping_up_by_card                  : 4 -> 3
balance_not_updated_after_bank_transfer <> transfer_not_received_by_recipient : 4 -> 5
         supported_cards_and_currencies <> visa_or_mastercard          : 4 -> 2
             topping_up_by_card <> transfer_into_account               : 4 -> 3
        pending_transfer <> transfer_not_received_by_recipient         : 4 -> 5
              beneficiary_not_allowed <> failed_tra